## Gradcam calculation

Grad-CAM was originally developed for CNNs in the computer-vision domain [[1]](http://cnnlocalization.csail.mit.edu/Zhou_Learning_Deep_Features_CVPR_2016_paper.pdf)
. Its recent adaptation to GCNs allows the identification of discriminative nodes in non-Euclidean graph structures beyond image space [[2]](https://link.springer.com/book/10.1007/978-3-030-00689-1). In this work, we follow the Grad-CAM strategy described in [[3]](https://www.sciencedirect.com/science/article/pii/S016503272300928X) to highlight brain regions that uniquely contribute to classification outcomes across male and female groups.


> Steps




Here, we compute Grad-CAM values at the input level of the GCN using a (Gradient × Input) strategy. First, the input feature matrix
𝑉 is set to require gradients, and the model performs a forward pass to generate class scores. The softmax probability corresponding to the MDD class is then selected, and gradients of this target score with respect to the input 𝑉 are obtained via backpropagation. These gradients capture how sensitive the prediction is to changes in each input element. To quantify contribution, the gradients are multiplied element-wise with the original input features, producing a Grad-CAM map that highlights the most influential input positions. Finally, the Grad-CAM values are averaged across the feature dimension to yield a contribution score for each ROI, indicating its importance in predicting the MDD class. This procedure is applied separately for models trained on male and female subjects to identify sex-specific discriminative brain regions.


In [ ]:
def calculate_grad_cam(model, data):
    data = data.to(device)
    model.zero_grad()
    data.x.requires_grad = True

    output, _ = model(data)

    scores = F.softmax(output, dim=1)
    print(scores.shape)
    print(scores[0])

    scores_mdd = scores[:, 1]
    print('scores = ',scores_mdd.shape)

    scores_mdd.backward(torch.ones_like(scores_mdd))

    gradients = data.x.grad
    print('gradients = ',gradients.shape)

    # Calculate Grad-CAM values
    grad_cam = gradients * data.x
    print('data.x = ',data.x.shape)
    print('gradcam = ',grad_cam.shape)

    contribution = torch.mean(grad_cam, dim=1)

    print('contribution = ',contribution.shape)

    return scores, grad_cam, contribution